In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.consumers = []

        Graph.get_default().variables.append(self)

In [3]:
class Placeholder:
    def __init__(self):
        self.consumers = []

        Graph.get_default().placeholders.append(self)

In [14]:
class Operation:
    def __init__(self, input_nodes=None):
        self.input_nodes = input_nodes if input_nodes is not None else []
        self.consumers = []
        self.inputs = None
        self.output = None

        for input_node in self.input_nodes:
            input_node.consumers.append(self)

        Graph.get_default().operations.append(self)

    def compute(self, *args):
        pass
    
    def gradient(self, gradient):
        pass

In [15]:
class Add(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x: float, y: float) -> float:
        return x + y
    
    def gradient(self, gradient):
        x, y = self.inputs
        
        gradient_x, gradient_y = gradient, gradient
        
        while np.ndim(gradient_x) > len(x.shape):
            gradient_x = np.sum(gradient_x, axis=0)
        for axis, size in enumerate(x.shape):
            if size == 1:
                gradient_x = np.sum(gradient_x, axis=axis, keepdims=True)

        while np.ndim(gradient_y) > len(y.shape):
            gradient_y = np.sum(gradient_y, axis=0)
        for axis, size in enumerate(y.shape):
            if size == 1:
                gradient_y = np.sum(gradient_y, axis=axis, keepdims=True)

        return [gradient_x, gradient_y]


class MatrixMultiplication(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        return x.dot(y)
    
    def gradient(self, gradient):
        x, y = self.inputs
        
        return [gradient.dot(y.T), x.T.dot(gradient)]


class Softmax(Operation):
    def __init__(self, logits):
        super().__init__([logits])

    def compute(self, logits: np.ndarray):
        return np.exp(logits) / np.sum(np.exp(logits), axis=1)[:, None]
    
    def gradient(self, gradient):
        softmax = self.output
        return (gradient - np.reshape(
            np.sum(gradient * softmax, 1),
            [-1, 1]
        )) * softmax


class Log(Operation):
    def __init__(self, x):
        super().__init__([x])

    def compute(self, x: np.ndarray):
        return np.log(x)
    
    def gradient(self, gradient):
        x, = self.inputs
        return gradient / x


class HadamardProduct(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x: np.ndarray, y: np.ndarray):
        return x * y
    
    def gradient(self, gradient):
        x, y = self.inputs
        
        return [gradient * y, gradient * x]


class MatrixSum(Operation):
    def __init__(self, x, axis: int = None):
        super().__init__([x])
        self.axis = axis

    def compute(self, x: np.ndarray):
        return np.sum(x, axis=self.axis)
    
    def gradient(self, gradient):
        x, = self.inputs

        output_shape = np.array(x.shape)
        output_shape[self.axis] = 1
        tile_scaling = x.shape // output_shape
        gradient = np.reshape(gradient, output_shape)
        return np.tile(gradient, tile_scaling)


class Negative(Operation):
    def __init__(self, x):
        super().__init__([x])

    def compute(self, x: float | np.ndarray):
        return -x
    
    def gradient(self, gradient):
        return -gradient


class Convolution(Operation):
    def __init__(self, x, _filter):
        super().__init__([x, _filter])
    
    def compute(self, x: np.ndarray, _filter: np.ndarray):
        x_height, x_width, *_ = x.shape
        filter_height, filter_width, *_ = _filter.shape
        result = np.zeros((x_height - filter_height + 1, x_width - filter_width + 1))
        
        for i in range(result.shape[0]):
            for j in range(result.shape[1]):
                result[i, j] = np.sum(x[i: i + filter_height, j: j + filter_width] * _filter)
        
        return result
    
    def gradient(self, gradient):
        input_x, input_filter = self.inputs
        input_height, input_width, *_ = input_x.shape
        input_filter_height, input_filter_width, input_filter_channels, *_ = input_filter.shape
        
        filter_gradient = np.zeros_like(input_filter)
        

In [16]:
class Graph:
    default = None

    def __init__(self):
        self.operations: list[Operation] = []
        self.placeholders: list[Placeholder] = []
        self.variables: list[Variable] = []

    @classmethod
    def get_default(cls):
        if cls.default is None:
            cls.default = cls()
        return cls.default

In [17]:
class Session:
    def run(self, operation, feed_dict=None):
        if feed_dict is None:
            feed_dict = {}

        for node in traverse_postorder(operation):
            if isinstance(node, Placeholder):
                node.output = feed_dict[node]
            elif isinstance(node, Variable):
                node.output = node.value
            elif isinstance(node, Operation):
                node.inputs = [input_node.output for input_node in node.input_nodes]

                node.output = node.compute(*node.inputs)

            if isinstance(node.output, list):
                node.output = np.array(node.output)

        return operation.output

In [18]:
def traverse_postorder(node):
    if isinstance(node, Operation):
        for input_node in node.input_nodes:
            yield from traverse_postorder(input_node)
    yield node

In [19]:
def compute_gradients(loss):
    gradient_table = {loss: 1}

    visited = set()
    queue = []
    visited.add(loss)
    queue.append(loss)

    while queue:
        node = queue.pop(0)

        if node != loss:
            gradient_table[node] = 0

            for consumer in node.consumers:
                upstream_gradient = gradient_table[consumer]
                local_gradient = consumer.gradient(upstream_gradient)

                if len(consumer.input_nodes) == 1:
                    gradient_table[node] += local_gradient

                else:
                    node_index_in_consumer_inputs = consumer.input_nodes.index(node)
                    downstream_gradient = local_gradient[node_index_in_consumer_inputs]
                    gradient_table[node] += downstream_gradient

        if hasattr(node, 'input_nodes'):
            for input_node in node.input_nodes:
                if not input_node in visited:
                    visited.add(input_node)
                    queue.append(input_node)

    return gradient_table

In [20]:
class GradientDescentOptimizer:
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def minimize(self, loss):
        learning_rate = self.learning_rate

        class MinimizationOperation(Operation):
            def compute(self):
                grad_table = compute_gradients(loss)

                for node in grad_table:
                    if isinstance(node, Variable):
                        grad = grad_table[node]
                        node.value -= learning_rate * grad

        return MinimizationOperation()

In [ ]:
class Layer:
    pass


class Linear(Layer):
    def __init__(self, input_size: int, output_size: int):
        self.weights = Variable(np.random.randn(output_size, input_size))
        self.bias = Variable(np.random.randn(output_size))
        self.expression = None
    
    def build_expression(self, input_variable: Variable | Operation | Placeholder):
        self.expression = Add(MatrixMultiplication(self.weights, input_variable), self.bias)    

In [22]:
X = Placeholder()
c = Placeholder()

# Initialize weights randomly
W = Variable(np.random.randn(2, 2))
b = Variable(np.random.randn(2))

red_points = np.random.randn(50, 2) - 2*np.ones((50, 2))
blue_points = np.random.randn(50, 2) + 2*np.ones((50, 2))

# Build perceptron
p = Softmax(Add(MatrixMultiplication(X, W), b))

# Build cross-entropy loss
J = Negative(MatrixSum(MatrixSum(HadamardProduct(c, Log(p)), axis=1)))

# Build minimization op
minimization_op = GradientDescentOptimizer(learning_rate=0.01).minimize(J)

# Build placeholder inputs
feed_dict = {
    X: np.concatenate((blue_points, red_points)),
    c:
        [[1, 0]] * len(blue_points)
        + [[0, 1]] * len(red_points)

}

# Create session
session = Session()

# Perform 100 gradient descent steps
for step in range(100):
    J_value = session.run(J, feed_dict)
    if step % 10 == 0:
        print("Step:", step, " Loss:", J_value)
    session.run(minimization_op, feed_dict)

# Print final result
W_value = session.run(W)
print("Weight matrix:\n", W_value)
b_value = session.run(b)
print("Bias:\n", b_value)

Step: 0  Loss: 105.45552948357856
Step: 10  Loss: 1.6071027418326638
Step: 20  Loss: 1.149579394968531
Step: 30  Loss: 0.9443880885592876
Step: 40  Loss: 0.8254483742083262
Step: 50  Loss: 0.746690417168796
Step: 60  Loss: 0.6900418734881323
Step: 70  Loss: 0.6469102020357418
Step: 80  Loss: 0.6126702690067443
Step: 90  Loss: 0.5846056511080009
Weight matrix:
 [[ 0.87893925 -1.23097457]
 [ 0.71675747 -2.62101925]]
Bias:
 [0.53091638 0.25638074]
